In [3]:
import os
import glob
import imageio
import random, shutil
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display
import librosa
import librosa.display
import pandas as pd
import seaborn as sns
import requests
import imageio
from zipfile import ZipFile
from PIL import Image, ImageOps, ImageShow
from torchvision.utils import save_image 

Pokemone sprites sourced from: https://pkmn.net/?action=content&page=viewpage&id=8644&parentsection=87
Includes sprites from BW, DPPt, FRLG, GSC, HGSS, Pinnball, RBYG, RSE. Removed Mystery Dunegon, Trozei, and GSC folders for poor quality. Removed Back sprites for consistency.

Utilizing AEGAN architechture from https://arxiv.org/abs/2004.05472,
code at: https://github.com/ConorLazarou/PokeGAN

Things changed:
- increased size of dataset to +9000 files
- removed transparency from .png files
- changed max batch size from 100 -> 285 (main.py: line 91) 

In [1]:
# Utils
def set_device():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device != "cuda":
        print("WARNING: For this notebook to perform best, "
          "if possible, in the menu under `Runtime` -> "
          "`Change runtime type.`  select `GPU` ")
    else:
        print("GPU is enabled in this notebook.")

    return device

def padding(img, expected_size):
    desired_size = expected_size
    delta_width = desired_size - img.size[0]
    delta_height = desired_size - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    
    new_img = Image.new(img.mode, (expected_size,expected_size), (255,255,255))
    new_img.paste(img, (pad_width, pad_height))
    #padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return new_img #ImageOps.expand(img, padding, fill = (255,255,255))


def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)#,fill = 255)

def remove_transparency(im, bg_colour=(255, 255, 255)):

    # Only process if image has transparency (http://stackoverflow.com/a/1963146)
    if im.mode in ('RGBA', 'LA') or (im.mode == 'P' and 'transparency' in im.info):

        # Need to convert to RGBA if LA format due to a bug in PIL (http://stackoverflow.com/a/1963146)
        alpha = im.convert('RGBA').split()[-1]

        # Create a new background image of our matt color.
        # Must be RGBA because paste requires both images have the same format
        # (http://stackoverflow.com/a/8720632  and  http://stackoverflow.com/a/9459208)
        bg = Image.new("RGBA", im.size, bg_colour + (255,))
        bg.paste(im, mask=alpha)
        return bg

    else:
        return im

#def iter_frames(im):
#    try:
#        i= 0
#        while 1:
#            im.seek(i)
#            imframe = im.copy()
#            if i == 0: 
#                palette = imframe.getpalette()
#            else:
#                imframe.putpalette(palette)
#            yield imframe
#            i += 1
#    except EOFError:
#        pass
#
#    for i, frame in enumerate(iter_frames(im)):
#        frame.save('test%d.png' % i,**frame.info)   

In [7]:
src_path = "C:\\Users\\ipzac\\Documents\\Project Data\\Pokemon Sprites\\Original Pokémon\\"
save_path = "C:\\Users\\ipzac\\Documents\\Project Data\\Pokemon Sprites\\Clean Sprites\\"
os.chdir(src_path)
template = np.zeros((96,96))
src_files = []
unique_ID = 0

for root,d, files in os.walk(src_path):
    for f in files:
        im = Image.open(root + '\\' + f)
        im = remove_transparency(im)
        #bw_im = ImageOps.grayscale(im)
        pad_im = padding(im, 96)
        pad_im.save(save_path + str(unique_ID) + ".png")
        #tensor_im = transforms.ToTensor()(pad_im).squeeze()
        #torch.save(tensor_im, save_path + str(unique_ID) + ".pt")
        unique_ID += 1
        #ImageShow.show(pad_im)

In [4]:
set_device()

GPU is enabled in this notebook.


'cuda'

In [ ]:
file_path = "C:\\Users\\ipzac\\Documents\\Personal-Projects\\Sprite Image Generation\\"
os.chdir(file_path)

%run main.py

Epoch 1; Elapsed time = 00:00:00s
101/286: G=5.363, E=0.669, Dx=0.033, Dz=0.712  Rx=0.643  Rz=0.651
Epoch 2; Elapsed time = 00:01:09s
101/286: G=8.437, E=0.683, Dx=0.002, Dz=0.712  Rx=0.208  Rz=0.254
Epoch 3; Elapsed time = 00:02:18s
101/286: G=6.799, E=0.695, Dx=0.185, Dz=0.705  Rx=0.159  Rz=0.142
Epoch 4; Elapsed time = 00:03:27s
101/286: G=5.820, E=0.695, Dx=0.058, Dz=0.704  Rx=0.146  Rz=0.111
Epoch 5; Elapsed time = 00:04:36s
101/286: G=5.575, E=0.698, Dx=0.110, Dz=0.701  Rx=0.144  Rz=0.096
Epoch 6; Elapsed time = 00:05:46s
101/286: G=5.079, E=0.700, Dx=0.100, Dz=0.698  Rx=0.143  Rz=0.088
Epoch 7; Elapsed time = 00:06:55s
101/286: G=5.074, E=0.699, Dx=0.111, Dz=0.699  Rx=0.142  Rz=0.085
Epoch 8; Elapsed time = 00:08:03s
101/286: G=4.967, E=0.700, Dx=0.114, Dz=0.697  Rx=0.140  Rz=0.084
Epoch 9; Elapsed time = 00:09:12s
101/286: G=4.747, E=0.702, Dx=0.114, Dz=0.696  Rx=0.140  Rz=0.080
Epoch 10; Elapsed time = 00:10:21s
101/286: G=4.570, E=0.702, Dx=0.129, Dz=0.695  Rx=0.138  Rz=0.082

101/286: G=2.700, E=0.701, Dx=0.306, Dz=0.689  Rx=0.127  Rz=0.062
Epoch 83; Elapsed time = 01:34:11s
101/286: G=2.650, E=0.701, Dx=0.251, Dz=0.688  Rx=0.130  Rz=0.062
Epoch 84; Elapsed time = 01:35:20s
101/286: G=2.584, E=0.702, Dx=0.272, Dz=0.687  Rx=0.130  Rz=0.059
Epoch 85; Elapsed time = 01:36:29s
101/286: G=2.629, E=0.706, Dx=0.300, Dz=0.686  Rx=0.128  Rz=0.061
Epoch 86; Elapsed time = 01:37:38s
101/286: G=2.622, E=0.706, Dx=0.266, Dz=0.685  Rx=0.129  Rz=0.062
Epoch 87; Elapsed time = 01:38:47s
101/286: G=2.610, E=0.707, Dx=0.296, Dz=0.685  Rx=0.128  Rz=0.060
Epoch 88; Elapsed time = 01:39:56s
101/286: G=2.620, E=0.704, Dx=0.272, Dz=0.687  Rx=0.128  Rz=0.061
Epoch 89; Elapsed time = 01:41:05s
101/286: G=2.577, E=0.704, Dx=0.272, Dz=0.686  Rx=0.126  Rz=0.059
Epoch 90; Elapsed time = 01:42:14s
101/286: G=2.638, E=0.703, Dx=0.294, Dz=0.687  Rx=0.129  Rz=0.061
Epoch 91; Elapsed time = 01:43:23s
101/286: G=2.570, E=0.702, Dx=0.263, Dz=0.687  Rx=0.128  Rz=0.060
Epoch 92; Elapsed time = 